In [1]:
!pip install --no-cache-dir --force-reinstall \
    "transformers==4.53.0" \
    "accelerate==1.8.1" \
    "trl==0.19.0" \
    "peft==0.10.0" \
    datasets           # 버전 고정 X → trl 과 호환되는 최신 설치

^C
ERROR: Operation cancelled by user

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import trl, transformers, accelerate
print("trl", trl.__version__)
print("transformers", transformers.__version__)
print("accelerate", accelerate.__version__)

trl 0.19.0
transformers 4.53.0
accelerate 1.8.1


# 데이터 살펴보기

In [3]:
import pandas as pd

url = 'https://raw.githubusercontent.com/sogmgm/sogm_dataset/main/event_dataset_for_ft.jsonl'
df = pd.read_json(url, lines=True, encoding='utf-8')
print(len(df))

3265


In [4]:

print(df.info())
df['text_category'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3265 entries, 0 to 3264
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   doc_id            3265 non-null   int64 
 1   doc_title         3265 non-null   object
 2   text              3265 non-null   object
 3   text_category     3265 non-null   object
 4   event_quantity    3265 non-null   int64 
 5   event_sentence1   3265 non-null   object
 6   event_sentence2   3152 non-null   object
 7   event_sentence3   938 non-null    object
 8   event_sentence4   310 non-null    object
 9   event_sentence5   127 non-null    object
 10  event_sentence6   52 non-null     object
 11  event_sentence7   24 non-null     object
 12  event_sentence8   9 non-null      object
 13  event_sentence9   3 non-null      object
 14  event_sentence10  2 non-null      object
dtypes: int64(2), object(13)
memory usage: 382.7+ KB
None


array(['부동산', '산업', '오피니언', '증권'], dtype=object)

In [5]:
# 데이터 출력
n= 5 # n번째 문서
print(f"===== {n} 번째 데이터 샘플 =====")
print(f"문서 ID: {df.iloc[n]['doc_id']}")
print(f"제목: {df.iloc[n]['doc_title']}")
print(f"카테고리: {df.iloc[n]['text_category']}")
print(f"이벤트 개수: {df.iloc[n]['event_quantity']}")

print("\n===== 본문 =====")
print(df.iloc[n]['text'])

print("\n=====이벤트 문장들 =====")
for i in range(1, 11):
    event_col = f'event_sentence{i}'
    if pd.notna(df.iloc[n][event_col]):
        print(f"이벤트 {i}: {df.iloc[n][event_col]}")


===== 5 번째 데이터 샘플 =====
문서 ID: 21943
제목: 목동6단지 안전진단 통과…재건축 탄력
카테고리: 부동산
이벤트 개수: 3

===== 본문 =====
목동6단지가 서울 양천구 목동신시가지 2만6000여 가구 중 최초로 재건축 정밀안전진단을 조건부로 통과했다. 정밀안전진단은 재건축을 위한 첫 관문이다. 지난주 목동 재건축에서 가장 큰 걸림돌이었던 1~3단지 종상향(용적률 50% 증가) 문제도 해결돼 이 일대 재건축사업이 급물살을 탈 전망이다. 다만 이번 안전진단이 조건부 통과이고 향후 정비구역 지정, 조합 설립 및 각종 인가 등 숱한 단계가 남아 있어 본격적으로 재건축이 추진되기까지 상당한 시일이 소요될 전망이다.  31일 양천구청은 목동신시가지6단지 재건축을 위한 정밀안전진단 결과가 'D등급'이라고 목동6단지 재건축추진준비위원회에 통보했다. 재건축 안전진단의 경우 A~C등급은 유지·보수(재건축 불가), D등급은 조건부 재건축(공공기관 검증 필요), E등급은 재건축 확정 판정으로 분류된다. 목동6단지는 D등급을 받은 만큼 향후 6개월간 국토교통부 산하 한국시설안전공단의 적정성 검토를 거쳐 최종 안전진단 결과를 통보받는다.  이번에 목동6단지는 종합평가 결과 51.22점을 받아 조건부 재건축 점수 기준(30점 초과~55점 이하)을 충족했다. 지난해 10월 안전진단을 통과하지 못한 송파구 방이동 올림픽선수촌아파트(이하 올선)는 58.61점으로 재건축 불가 판정을 받았는데 이와 비교해보면 7점 넘게 낮은 점수를 받았다.    두 아파트 단지를 비교해 보면 목동6단지는 항목 4개 중 2개(주거 환경·비용 분석)에선 올선에 비해 점수가 높았지만, 나머지 2개 항목(건축 마감 및 설비노후도·구조안전성)은 점수가 낮았다. 다시 말해 목동은 주거 환경은 올선에 비해 좋지만 노후화돼 있고 구조적으로 안전하지 않다는 것이다. 신종섭 목동6단지 재건축추진준비위원장은 "목동의 경우 아파트 아래에 철근 기둥만 박고, 철근 기둥을 받치는 지지대가 없어 지진이 멀

In [6]:
import json

def create_structured_chat_data(df):
    training_data = []

    for idx, row in df.iterrows():
        # 이벤트 문장들 수집
        event_sentences = []
        for i in range(1, 11):
            event_col = f'event_sentence{i}'
            if pd.notna(row[event_col]):
                event_sentences.append(row[event_col])

        # 구조화된 시스템 프롬프트
        system_prompt = """당신은 뉴스 텍스트를 분석하여 카테고리를 분류하고 주요 핵심 이벤트들을 추출하는 전문 분석 시스템입니다.
주어진 텍스트를 분석하여 반드시 파이썬의 dictionary 형식으로 결과를 작성하십시오.
큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다.
아래 dictionary에서 각 value는 지시사항에 해당합니다. 지시사항을 그대로 적지 마시고, 해당 지시사항에 따라 적절한 value를 채워넣으십시오.

분석 결과는 다음 형식으로 작성하십시오:

답변:
{"category": "텍스트의 카테고리를 ['부동산', '산업', '오피니언', '증권'] 중 하나로 분류하여 작성하십시오",
"event_count": "텍스트 내에서 발견된 핵심 이벤트의 개수를 정수로 작성하십시오(증감, 변화, 변동이 나오는 구절을 중점적으로 보세요.)",
"events": ["텍스트에서 추출한 각 핵심 이벤트를 요약하는 문장들을 파이썬 문자열 리스트 형태로 작성하십시오. 원문의 핵심 내용을 그대로 유지하되 한 문장으로 간결하게 작성하십시오"]}"""

        # 사용자 프롬프트
        user_prompt = f"""다음 뉴스 텍스트를 분석해주세요:

{row['text']}"""

        # 어시스턴트 응답 (JSON 형식)
        events_list = [event.strip() for event in event_sentences]

        assistant_response = f"""{{"category": "{row['text_category']}","event_count": {row['event_quantity']},"events": {json.dumps(events_list, ensure_ascii=False)}}}"""

        sample = {
                      "doc_id": row['doc_id'],
                      "system_prompt": system_prompt,
                      "user_prompt": user_prompt,
                      "assistant": assistant_response
                  }

        training_data.append(sample)

    return training_data

# 데이터 함수 실행
training_data = create_structured_chat_data(df)

# 첫 번째 샘플 확인
print(" ===== 시스템 프롬프트  =====")
print(training_data[0]['system_prompt'])
print("\n===== 사용자 프롬프트  =====")
print(training_data[0]['user_prompt'][:200] + "...")
print("\n===== 어시스턴트 응답 =====")
print(training_data[0]['assistant'])

 ===== 시스템 프롬프트  =====
당신은 뉴스 텍스트를 분석하여 카테고리를 분류하고 주요 핵심 이벤트들을 추출하는 전문 분석 시스템입니다.
주어진 텍스트를 분석하여 반드시 파이썬의 dictionary 형식으로 결과를 작성하십시오.
큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다.
아래 dictionary에서 각 value는 지시사항에 해당합니다. 지시사항을 그대로 적지 마시고, 해당 지시사항에 따라 적절한 value를 채워넣으십시오.

분석 결과는 다음 형식으로 작성하십시오:

답변:
{"category": "텍스트의 카테고리를 ['부동산', '산업', '오피니언', '증권'] 중 하나로 분류하여 작성하십시오",
"event_count": "텍스트 내에서 발견된 핵심 이벤트의 개수를 정수로 작성하십시오(증감, 변화, 변동이 나오는 구절을 중점적으로 보세요.)",
"events": ["텍스트에서 추출한 각 핵심 이벤트를 요약하는 문장들을 파이썬 문자열 리스트 형태로 작성하십시오. 원문의 핵심 내용을 그대로 유지하되 한 문장으로 간결하게 작성하십시오"]}

===== 사용자 프롬프트  =====
다음 뉴스 텍스트를 분석해주세요:

대통령 선거 후 수도권을 중심으로 한 부동산시장이 조금씩 바닥을 다지며 다시 상승을 준비하는 모습이 곳곳에서 포착되고 있다. 윤석열 대통령 당선인의 재건축·재개발 공약으로 서울에서 재건축 단지가 많은 지역 아파트값은 하락폭을 줄이고 있고, 일선 부동산 공인중개소에서는 매물을 거둬들이는 이가 점차 늘고 있는 것으로 나타났다...

===== 어시스턴트 응답 =====
{"category": "부동산","event_count": 2,"events": ["17일 한국부동산원에 따르면 서울의 3월 둘째 주(14일 기준) 아파트 매매가격지수는 전주 대비 0.02% 하락했다.", "경기 지역 역시 재건축 대상 아파트가 많은 고양(-0.02%→0%)이 보합으로 돌아섰고

In [7]:
def format_data(sample):
    return {
        "messages": [
            {
                "role": "system",
                "content": sample["system_prompt"],
            },
            {
                "role": "user",
                "content": sample["user_prompt"],
            },
            {
                "role": "assistant",
                "content": str(sample["assistant"])
            },
        ],
    }

# 전체 데이터에 format_data 적용
formatted_data = [format_data(sample) for sample in training_data]

# 확인
print("\n ===== 포맷된 데이터 샘플  =====")
formatted_data[0]


 ===== 포맷된 데이터 샘플  =====


{'messages': [{'role': 'system',
   'content': '당신은 뉴스 텍스트를 분석하여 카테고리를 분류하고 주요 핵심 이벤트들을 추출하는 전문 분석 시스템입니다.\n주어진 텍스트를 분석하여 반드시 파이썬의 dictionary 형식으로 결과를 작성하십시오.\n큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다.\n아래 dictionary에서 각 value는 지시사항에 해당합니다. 지시사항을 그대로 적지 마시고, 해당 지시사항에 따라 적절한 value를 채워넣으십시오.\n\n분석 결과는 다음 형식으로 작성하십시오:\n\n답변:\n{"category": "텍스트의 카테고리를 [\'부동산\', \'산업\', \'오피니언\', \'증권\'] 중 하나로 분류하여 작성하십시오",\n"event_count": "텍스트 내에서 발견된 핵심 이벤트의 개수를 정수로 작성하십시오(증감, 변화, 변동이 나오는 구절을 중점적으로 보세요.)",\n"events": ["텍스트에서 추출한 각 핵심 이벤트를 요약하는 문장들을 파이썬 문자열 리스트 형태로 작성하십시오. 원문의 핵심 내용을 그대로 유지하되 한 문장으로 간결하게 작성하십시오"]}'},
  {'role': 'user',
   'content': '다음 뉴스 텍스트를 분석해주세요:\n\n대통령 선거 후 수도권을 중심으로 한 부동산시장이 조금씩 바닥을 다지며 다시 상승을 준비하는 모습이 곳곳에서 포착되고 있다. 윤석열 대통령 당선인의 재건축·재개발 공약으로 서울에서 재건축 단지가 많은 지역 아파트값은 하락폭을 줄이고 있고, 일선 부동산 공인중개소에서는 매물을 거둬들이는 이가 점차 늘고 있는 것으로 나타났다. 다만 전문가들은 아직 부동산과 관련해 구체적인 정책들이 나오지 않았기에 매도자들은 좀 더 지켜볼 필요가 있다는 반응이다.     17일 한국부동산원에 따르면 서울의 3월 둘째 주(14일 기준) 아파트 매매가격지수는 전주 대비 0.02

In [8]:
# 모델과 토크나이저 로드
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-1.7B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-1.7B")

# pad_token 설정 (필요한 경우)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("모델과 토크나이저 로드 완료!")
print(f"Tokenizer vocab size: {tokenizer.vocab_size}")
print(f"Model parameters: {model.num_parameters():,}")

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

모델과 토크나이저 로드 완료!
Tokenizer vocab size: 151643
Model parameters: 1,720,574,976


In [9]:
# 전체 데이터에 템플릿 적용
texts = []
for sample in formatted_data:
    text = tokenizer.apply_chat_template(
        sample["messages"],
        tokenize=False,
        add_generation_prompt= False,
        chat_template_kwargs= {"enable_thinking": False}

    )
    texts.append(text)

print(f"템플릿 적용 완료: {len(texts)}개 샘플")
print(f"첫 번째 샘플 길이: {len(texts[0])}")
print(texts[0])

템플릿 적용 완료: 3265개 샘플
첫 번째 샘플 길이: 2684
<|im_start|>system
당신은 뉴스 텍스트를 분석하여 카테고리를 분류하고 주요 핵심 이벤트들을 추출하는 전문 분석 시스템입니다.
주어진 텍스트를 분석하여 반드시 파이썬의 dictionary 형식으로 결과를 작성하십시오.
큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다.
아래 dictionary에서 각 value는 지시사항에 해당합니다. 지시사항을 그대로 적지 마시고, 해당 지시사항에 따라 적절한 value를 채워넣으십시오.

분석 결과는 다음 형식으로 작성하십시오:

답변:
{"category": "텍스트의 카테고리를 ['부동산', '산업', '오피니언', '증권'] 중 하나로 분류하여 작성하십시오",
"event_count": "텍스트 내에서 발견된 핵심 이벤트의 개수를 정수로 작성하십시오(증감, 변화, 변동이 나오는 구절을 중점적으로 보세요.)",
"events": ["텍스트에서 추출한 각 핵심 이벤트를 요약하는 문장들을 파이썬 문자열 리스트 형태로 작성하십시오. 원문의 핵심 내용을 그대로 유지하되 한 문장으로 간결하게 작성하십시오"]}<|im_end|>
<|im_start|>user
다음 뉴스 텍스트를 분석해주세요:

대통령 선거 후 수도권을 중심으로 한 부동산시장이 조금씩 바닥을 다지며 다시 상승을 준비하는 모습이 곳곳에서 포착되고 있다. 윤석열 대통령 당선인의 재건축·재개발 공약으로 서울에서 재건축 단지가 많은 지역 아파트값은 하락폭을 줄이고 있고, 일선 부동산 공인중개소에서는 매물을 거둬들이는 이가 점차 늘고 있는 것으로 나타났다. 다만 전문가들은 아직 부동산과 관련해 구체적인 정책들이 나오지 않았기에 매도자들은 좀 더 지켜볼 필요가 있다는 반응이다.     17일 한국부동산원에 따르면 서울의 3월 둘째 주(14일 기준) 아파트 매매가격지수는 전주 대비 0.02% 하락했다. 8주 연속 떨어졌지만 지난

In [10]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import Dataset
import torch

In [11]:
def simple_tokenize_function(examples):
    result = tokenizer(
        examples["text"],
        truncation=True,
        max_length=8192,
        padding=False,
        return_overflowing_tokens=False,
    )

    # labels 초기화
    result["labels"] = []

    for i, text in enumerate(examples["text"]):
        input_ids = result["input_ids"][i]
        labels = [-100] * len(input_ids)

        # <|im_start|>assistant\n 토큰들 찾기 (전체 시퀀스)
        assistant_tokens = tokenizer.encode("<|im_start|>assistant\n", add_special_tokens=False)

        # assistant 시작 위치 찾기
        for j in range(len(input_ids) - len(assistant_tokens) + 1):
            if input_ids[j:j+len(assistant_tokens)] == assistant_tokens:
                # assistant\n 다음부터 학습 시작
                start_idx = j + len(assistant_tokens)

                # <|im_end|> 토큰 찾기
                im_end_token = tokenizer.encode("<|im_end|>", add_special_tokens=False)[0]
                for k in range(start_idx, len(input_ids)):
                    if input_ids[k] == im_end_token:
                        end_idx = k
                        break
                else:
                    end_idx = len(input_ids)

                # assistant 답변 부분만 학습하도록 설정
                labels[start_idx:end_idx] = input_ids[start_idx:end_idx]
                break

        result["labels"].append(labels)

    return result

# 다시 토크나이징

# texts 리스트를 dataset으로 변환
dataset = Dataset.from_dict({"text": texts})
tokenized_dataset = dataset.map(simple_tokenize_function, batched=True)
print(f"토크나이징 완료: {len(tokenized_dataset)}개 샘플")

Map:   0%|          | 0/3265 [00:00<?, ? examples/s]

토크나이징 완료: 3265개 샘플


In [12]:
# 토크나이징 후 데이터 확인
print("===== 토크나이징 후 데이터셋 정보 =====")
print(tokenized_dataset)
print(f"컬럼: {tokenized_dataset.column_names}")
print(f"첫 번째 샘플 input_ids 길이: {len(tokenized_dataset[0]['input_ids'])}")

# 길이 분포 확인
lengths = [len(sample['input_ids']) for sample in tokenized_dataset]
print(f"토큰 길이 - 최소: {min(lengths)}, 최대: {max(lengths)}, 평균: {sum(lengths)/len(lengths):.1f}")

===== 토크나이징 후 데이터셋 정보 =====
Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 3265
})
컬럼: ['text', 'input_ids', 'attention_mask', 'labels']
첫 번째 샘플 input_ids 길이: 1685
토큰 길이 - 최소: 619, 최대: 6885, 평균: 1394.5


In [13]:
# 첫 번째 샘플 선택
sample = tokenized_dataset[0]
print("샘플 하나를 자세히 분석")
print("=" * 60)

print("1. text (원본 텍스트)")
print(f"   문자 수: {len(sample['text']):,} 문자")
print(f"   내용:")
print(f"   {sample['text'][:200]}...")
print(f"   ...{sample['text'][-100:]}")
print("=" * 60)

print("2. input_ids (토큰 ID 숫자들)")
print(f"   토큰 개수: {len(sample['input_ids'])} 개")
print(f"   시작 부분: {sample['input_ids'][:15]}")
print(f"   끝 부분: {sample['input_ids'][-15:]}")
print("=" * 60)

print("3. attention_mask (어느 토큰에 집중할지)")
print(f"   길이: {len(sample['attention_mask'])} (input_ids와 같아야 함)")
print(f"   시작: {sample['attention_mask'][:10]} (모두 1이면 정상)")
print(f"   끝: {sample['attention_mask'][-10:]}")
print(f"   모든 값이 1? {all(x == 1 for x in sample['attention_mask'])}")
print("   → 1=실제토큰, 0=패딩토큰 (현재는 패딩 없음)")
print("=" * 60)

print("4. labels (모델이 학습할 정답)")
print(f"   길이: {len(sample['labels'])} (input_ids와 같아야 함)")
print(f"   현재 input_ids와 동일? {sample['input_ids'] == sample['labels']}")
print(f"   시작: {sample['labels'][:10]}")
print(f"   끝: {sample['labels'][-10:]}")
print("=" * 60)

if sample['input_ids'] == sample['labels']:
    print("문제 발견!")
    print("   현재는 사용자 질문까지 학습하고 있어요!")
    print("   assistant 답변 부분만 학습해야 합니다.")
else:
    print("올바른 설정")
    print("   -100인 부분은 학습 안 함 (사용자 질문)")
    print("   실제 숫자인 부분만 학습 (assistant 답변)")

샘플 하나를 자세히 분석
1. text (원본 텍스트)
   문자 수: 2,684 문자
   내용:
   <|im_start|>system
당신은 뉴스 텍스트를 분석하여 카테고리를 분류하고 주요 핵심 이벤트들을 추출하는 전문 분석 시스템입니다.
주어진 텍스트를 분석하여 반드시 파이썬의 dictionary 형식으로 결과를 작성하십시오.
큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다.
아...
   ...", "경기 지역 역시 재건축 대상 아파트가 많은 고양(-0.02%→0%)이 보합으로 돌아섰고, 성남 분당은 전주와 같은 하락폭(-0.01%)을 기록했다."]}<|im_end|>

2. input_ids (토큰 ID 숫자들)
   토큰 개수: 1685 개
   시작 부분: [151644, 8948, 198, 64795, 82528, 33704, 5140, 231, 112, 24897, 10764, 44104, 53189, 18411, 128618]
   끝 부분: [135257, 4080, 15, 13, 15, 16, 11334, 17877, 54116, 49664, 128836, 1189, 13989, 151645, 198]
3. attention_mask (어느 토큰에 집중할지)
   길이: 1685 (input_ids와 같아야 함)
   시작: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] (모두 1이면 정상)
   끝: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
   모든 값이 1? True
   → 1=실제토큰, 0=패딩토큰 (현재는 패딩 없음)
4. labels (모델이 학습할 정답)
   길이: 1685 (input_ids와 같아야 함)
   현재 input_ids와 동일? False
   시작: [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
   끝: [16, 11334, 17877, 541

# Train! 

In [22]:
from trl import SFTConfig, SFTTrainer



# LoRA 설정
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "v_proj"],
    task_type="CAUSAL_LM"
)

# 모델에 LoRA 적용
model = get_peft_model(model, peft_config)



# 3) SFTConfig  ← 여기서 max_seq_length, packing 지정
sft_cfg = SFTConfig(
    output_dir="./qwen3-lora-ft",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    optim="adamw_torch",
    bf16=True,
    gradient_checkpointing=True,
    logging_steps=50,
    save_steps=50,
    save_total_limit=3,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="linear",
    report_to="none",
    remove_unused_columns=False,
    weight_decay=0.01,
    max_seq_length = 8192,
    packing        = False,
)


# 5) SFTTrainer
trainer = SFTTrainer(
    model             = model,
    args              = sft_cfg,           # ← SFTConfig 객체
    train_dataset     = tokenized_dataset,
    eval_dataset      = None,
    processing_class  = tokenizer,         # ← tokenizer 자리
)
model.enable_input_require_grads()

trainer.train_dataset = trainer.train_dataset.shuffle(seed=42)
trainer.train()

저장
trainer.model.save_pretrained("./qwen3-lora-ft-final")
print("훈련 완료! 모델 저장됨.")

Truncating train dataset:   0%|          | 0/3265 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [23]:
import sys, trl, inspect, importlib, pkg_resources, pathlib
print("Python :", sys.executable)
print("trl     :", trl.__version__)
print("trl path:", pathlib.Path(trl.__file__).parent)
print("SFTTrainer sig:", inspect.signature(trl.SFTTrainer.__init__))

Python : /usr/bin/python
trl     : 0.19.0
trl path: /usr/local/lib/python3.10/dist-packages/trl
SFTTrainer sig: (self, model: Union[str, torch.nn.modules.module.Module, transformers.modeling_utils.PreTrainedModel], args: Union[trl.trainer.sft_config.SFTConfig, transformers.training_args.TrainingArguments, NoneType] = None, data_collator: Optional[transformers.data.data_collator.DataCollator] = None, train_dataset: Union[datasets.arrow_dataset.Dataset, datasets.iterable_dataset.IterableDataset, NoneType] = None, eval_dataset: Union[datasets.arrow_dataset.Dataset, dict[str, datasets.arrow_dataset.Dataset], NoneType] = None, processing_class: Union[transformers.tokenization_utils_base.PreTrainedTokenizerBase, transformers.image_processing_utils.BaseImageProcessor, transformers.feature_extraction_utils.FeatureExtractionMixin, transformers.processing_utils.ProcessorMixin, NoneType] = None, compute_loss_func: Optional[Callable] = None, compute_metrics: Optional[Callable[[transformers.trainer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: trl 0.19.0
Uninstalling trl-0.19.0:
  Successfully uninstalled trl-0.19.0
Found existing installation: transformers 4.53.0
Uninstalling transformers-4.53.0:
  Successfully uninstalled transformers-4.53.0
Found existing installation: accelerate 1.8.1
Uninstalling accelerate-1.8.1:
  Successfully uninstalled accelerate-1.8.1
Note: you may need to restart the kernel to use updated packages.


In [16]:
trainer.model.save_pretrained("./qwen3-lora-ft-final")

# 평가

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
# 1. 토크나이저와  모델 로드

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-1.7B")

base= AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-1.7B",
    torch_dtype="auto",
    device_map="auto"
).eval()                     


lora = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-1.7B",
    torch_dtype="auto",
    device_map="auto"
)
lora = PeftModel.from_pretrained(
    lora,
    "./qwen3-lora-ft-final",
    device_map="auto",
    torch_dtype="auto"
).eval()

# 3. 시스템 프롬프트 
system_prompt = """당신은 뉴스 텍스트를 분석하여 카테고리를 분류하고 주요 핵심 이벤트들을 추출하는 전문 분석 시스템입니다.
주어진 텍스트를 분석하여 반드시 파이썬의 dictionary 형식으로 결과를 작성하십시오.
큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다.
아래 dictionary에서 각 value는 지시사항에 해당합니다. 지시사항을 그대로 적지 마시고, 해당 지시사항에 따라 적절한 value를 채워넣으십시오.

분석 결과는 다음 형식으로 작성하십시오:

답변:
{"category": "텍스트의 카테고리를 ['부동산', '산업', '오피니언', '증권'] 중 하나로 분류하여 작성하십시오",
"event_count": "텍스트 내에서 발견된 핵심 이벤트의 개수를 정수로 작성하십시오(증감, 변화, 변동이 나오는 구절을 중점적으로 보세요.)",
"events": ["텍스트에서 추출한 각 핵심 이벤트를 요약하는 문장들을 파이썬 문자열 리스트 형태로 작성하십시오. 원문의 핵심 내용을 그대로 유지하되 한 문장으로 간결하게 작성하십시오"]}"""

# 4. 테스트 데이터 3개 
test_texts = [
    # 1. 부동산 카테고리
    """ 최대 수혜지역이라고요? 분위기 좀 달라지긴 했지만, 그게 꼭 6·27 대출 규제 때문만은 아니죠."

지난 19일 서울 도봉구 창동역 인근 한 공인중개사는 최근 시장 분위기를 묻는 질문에 이같이 말했다. 도봉구는 창동 차량기지 복합개발, GTX-C 등 대형 교통 호재에 힘입어 올해 들어 아파트값이 상승세로 전환됐고, 일부 단지에서는 실거래가 회복 조짐도 나타나고 있다. 반면 6·27대책 발표 직후 상승세 강했던 금관구(금천·관악·구로)는 매수·매도인 모두 지켜보기에 들어간 모습이다.

■도봉구 상승세…실거래가 회복 조짐

6·27 대출 규제 시행 3주차에 접어들면서 서울 외곽 지역을 중심으로 시장의 온도차가 뚜렷해지고 있다. 한국부동산원 주간 아파트가격 동향에 따르면 7월 2주(15일 기준) 서울 아파트값은 0.19% 올라 전주(0.29%)보다 오름폭이 둔화됐다. 다만 도봉구(0.05%→0.06%)와 중구(0.16%→0.18%)는 상승폭이 소폭 확대됐다. 도봉구의 한 중개업소 관계자는 "급매가 거의 소진된 상태에서 전고점에 근접한 거래도 하나둘 나타나고 있다"며 "전용 84㎡, 165㎡ 등 다양한 평형에 실수요자와 일부 투자자의 문의가 이어지고 있다"고 전했다. 이어 "6·27 대책 이후 도봉구처럼 상대적으로 대출 여건이 유지되는 중저가 지역에 관심이 모이고 있는 건 사실이지만, 지금 시장을 움직이는 건 결국 개발 기대감에 기반한 유동성 있는 수요자들"이라고 설명했다.

서울부동산정보광장에 따르면 도봉구 아파트 거래량은 5월 155건, 6월 289건으로 늘었지만, 7월 들어서는 20일 기준 51건에 머물고 있다. 규제 여파로 거래량은 다소 숨고르기에 들어간 양상이지만 가격 상승세는 계속될 것으로 전망됐다.

도봉구의 또 다른 중개업소 대표는 "이미 작년에 바닥을 찍었다는 인식이 확산되면서 실수요자 문의가 늘고 있고, 호가도 빠르게 회복되고 있다"며 "GTX-C, 창동 차량기지 개발 등 향후 호재가 여전한 만큼, 지금은 회복의 시작일 뿐이라는 얘기도 나온다"고 했다.

■금관구 숨고르기…거래량 감소 뚜렷

반면 6·27 대책 직후 풍선효과 기대감이 부각됐던 '금관구'는 오름세가 한풀 꺾인 분위기다. 같은 기간 금천구는 0.09%→0.07%, 관악구는 0.19%→0.15%, 구로구는 0.18%→0.12%로 상승폭이 일제히 축소됐다.

금천구 시흥동의 한 중개업소 관계자는 "호가는 잠깐 반짝했지만 지금은 문의가 다시 줄어든 상황"이라며 "갭투자자든 실수요자든 모두 관망 중"이라고 말했다. 관악구의 한 공인중개사도 "전세보증금이 낮은 구축 소형 위주로 간헐적인 갭투자 수요는 있었지만, 지금은 전세금도 안 오르고 매수도 끊긴 상태"라고 분위기를 전했다.

서울부동산정보광장에 따르면 금천구 아파트 거래량은 5월 74건에서 6월 135건으로 증가한 뒤, 7월(20일 기준)에는 25건에 그쳤다. 관악구는 같은 기간 222건→294건→54건, 구로구는 678건→462건→84건으로 줄며 거래 감소세가 뚜렷하다.

개봉동의 중개업소 관계자는 "급매라고는 해도 1000만~2000만원 낮춘 수준인데, 그런 물건도 상반기에 이미 빠졌다"며 "매수자는 관망세, 매도자는 버티기에 들어간 상황"이라고 전했다. 결론적으로 매물 자체가 줄어들면서 거래량도 감소한 위축된 시장 흐름을 보이고 있어, 풍선효과가 나타난다고 보기는 어렵다는 평가다.

부동산업계 한 전문가는 "서울 외곽 지역은 규제 변화에 민감하게 반응하는 편이지만, 신축 비중이나 입지 여건, 실수요자의 선호도에 따라 시장 흐름은 엇갈릴 수밖에 없다"며 "현재는 거래 절벽에 가까운 상태로 '풍선효과' 역시 일부 지역에 국한된 단기 흐름에 그칠 가능성이 높다"고 진단했다.",
""",    
    # 2. 산업 카테고리
    """
    한일시멘트가 자회사 한일현대시멘트 흡수 합병을 통해 경영효율화에 나서면서 시멘트업계에 구조·사업개편 가능성이 제기된다. 국내 시멘트 산업이 '저성장 고비용'의 구조적 위험에 빠진데 따른 영향이라는 분석이다. 다만 한일시멘트를 제외한 주요 기업들은 우선 신중모드로 접근할 전망이다.

20일 관련 업계에 따르면 한일시멘트는 최근 이사회를 열고 77.78% 지분을 보유한 한일현대시멘트를 오는 11월 1일부로 흡수 합병하기로 의결했다. 양사는 2017년부터 단계적으로 통합 작업을 진행해왔다. 이번 합병으로 상장사 이중 구조를 해소하고 중복투자·외부비용 절감을 통한 경영 효율화에 나선다는 방침이다. 한일시멘트 관계자는 "경기가 부침이 있는 만큼 설비 등 인프라 활용해 탄력적 대응이 가능할 것"이라고 말했다.

이미 시멘트 업계의 경영상황은 악화일로다. 국토교통부에 따르면 지난 5월 건설착공면적은 전월 대비 26.9% 감소한 620만㎡ 수준에 그쳤다. 착공면적은 통상 건설경기의 선행지표로 여겨진다. 시멘트 출하량도 급감했다. 올해 1·4분기 시멘트 내수는 전년 동기 대비 21.8% 줄어든 812만t으로, 최근 5년 내 최저치를 기록했다. 2023년 1·4분기 1201만t 대비 2년 새 32.4%가 줄어든 셈이다.

그럼에도 업계 전반의 대응은 신중한 모습이다. 한일을 제외한 주요 업체들은 합병 혹은 조직개편 계획 여부 등은 없다는 입장이다. 쌍용C&E 관계자는 "경기가 좋지 않지만 수출 덕분에 그나마 나은 상황으로 당분간 특별한 계획 없다"고 말했다. 성신양회, 한라시멘트 관계자 역시 "거래처 및 비용 절감에 집중하고 있을 뿐, 지배구조 개편 등은 고려하고 있지 않다"고 했다.

업계는 현재 위기를 수요 부진과 고정비 부담의 이중고로 보고 있다. 시멘트 산업은 설비 비중이 높아 수요가 줄어도 일정 수준의 가동과 인력 유지를 지속해야 하기 때문이다. 게다가 최근 몇 년간 산업용 전기요금 인상, 탄소배출권 비용 증가, 폐기물 처리단가 상승 등 외부 비용도 커졌다.

이 가운데 업계는 환경·사회·지배구조(ESG) 전환에도 동시에 대응 중이다. 시멘트 제조 시 온실가스 배출량이 많다는 점에서, 탄소중립 압력이 거세져 온 탓이다. 이에 오는 2030년까지 2018년 대비 12%, 2050년까지 53%의 온실가스 감축을 목표로 설정하고 있다. 이를 위해 석회석 대신 비탄산염 원료를 활용한 저탄소 클링커 기술, 폐합성수지·바이오매스 등 순환자원 연료 전환 기술 개발 등에 집중하고 있다.

한일시멘트의 이번 결정은 불황기의 생존 해법으로 지배구조 단순화 및 비용 최적화의 가능성을 제기했다는 분석이 나온다.

업계 관계자는 "내수 의존도가 큰 기업일수록 버틸 수 있는 시간이 길지 않다"며 "누가 먼저 구조를 바꾸고, 얼마나 오래버티느냐가 관건"이라고 말했다.
    """,
    
    # 3. 증권 카테고리
    """ (서울=연합뉴스) 임화섭 기자 = 인공지능(AI) 분야 경쟁으로 '매그니피센트 세븐'(미국의 7대 테크 대기업·The Magnificent Seven) 사이에서도 주가 차별화가 심화되고 있다고 미국 일간 월스트리트저널(WSJ)이 20일(현지시간) 분석했다.

테크업계를 주름잡는 대기업들이며 각종 주가지수에서도 큰 비중을 차지하는 아마존닷컴(이하 아마존), 알파벳, 애플, 메타플랫폼스(이하 메타), 마이크로소프트(이하 MS), 엔비디아, 테슬라 등 7개 기업의 주가 흐름이 AI 분야 성과에 따라 크게 갈리는 경향이 최근 뚜렷해졌다는 것이다.

대표적인 주가지수인 S&P 500에서 이들 7개 업체가 차지하는 비중은 35%에 이른다.

최근 모건스탠리 보고서에 따르면 이 업체들의 2분기 순이익은 전년 동기 대비 14% 증가할 것으로 전망되며, 이는 나머지 493개 S&P 500 기업의 순이익이 전년 동기 대비 3% 감소할 것으로 예상되는 전반적 증시 분위기와 비교하면 훨씬 좋다.

이들 중 알파벳을 제외한 6개 업체들은 최근 주가가 향후 1년간 예상 순이익의 25배를 넘었으며, 이는 최근 S&P 500 기업의 평균 주가수익비율(PER) 22.35보다 높다.

올해 들어 엔비디아, 메타, MS의 주가는 약 20% 이상 올랐으나, 이와 대조적으로 애플은 16% 하락했고 알파벳도 2% 내렸고 테슬라는 18%가 빠졌다.

AI 스타트업 '앤스로픽'에 투자한 아마존은 관세 등의 영향으로 전자상거래 부문 사업에 타격이 상당히 큰 가운데서도 3% 올랐다.

알파벳과 테슬라는 오는 23일에, 메타, MS, 애플은 그 다음 주에 분기 실적을 발표할 예정이다.

해리스 파이낸셜 그룹의 매니징 파트너인 제이미 콕스는 WSJ에 "(테크 대기업들 사이에서 실적에 따라 주가 흐름이 차별화되는 것은) 불가피한 일이었다. 다른 일들을 하고 있기 때문에 영원히 서로 똑같은 (주가) 흐름이 계속되는 것은 불가능하다"며 "이제 승자와 패자의 계층분화가 왔다"고 평가했다.

2023년에 이 7개 업체를 묶어서 '매그니피센트 세븐'이라는 표현을 쓰기 시작한 뱅크오브아메리카 글로벌 리서치의 투자전략담당 책임자 마이클 하트넛은 이들 업체를 묶어서 본 중요한 이유 중 하나는 이들이 AI 분야의 선봉에 있기 때문이라고 말한 바 있다.

그러나 투자회사 스피어의 창립자 겸 투자책임자인 이바나 델레브스카는 "지금은 펀더멘털에서 (7개 업체들 사이에) 상당히 큰 차이가 보이고 있다"고 말했다.
    """
]
def generate_output(model, text, system_prompt):
    # messages 형식으로 프롬프트 구성 (system + user)
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"다음 뉴스 텍스트를 분석해주세요:\n\n{text}"}
    ]
    
    # apply_chat_template 사용 (thinking 끄기: enable_thinking=False)
    prompt_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False  # thinking 모드 OFF (여기서 변경!)
    )
    
    # 입력 토큰화
    inputs = tokenizer([prompt_text], return_tensors="pt").to(model.device)
    
    # generate (deterministic으로 설정)
    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_length=8192,  # 최대 길이
            temperature=0.5,  # deterministic (랜덤성 제거)
            top_p=1.0,
            do_sample=False   # 샘플링 off
        )
    
    # 출력 디코딩 (thinking이 없을 테니 전체 출력)
    output_ids = generated_ids[0][len(inputs.input_ids[0]):]  # 생성된 부분만
    decoded_output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    
    # 만약 thinking 태그가 남아 있으면 파싱 (안전장치, False로 하면 불필요)
    try:
        # </think> 토큰 ID (Qwen 공식: 151668, 모델에 따라 확인 필요)
        think_end_token = 151668  # Qwen의 </think> 토큰 ID
        index = len(output_ids.tolist()) - output_ids.tolist()[::-1].index(think_end_token)
    except ValueError:
        index = 0  # thinking 없으면 0
    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip()
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip()
    
    # thinking이 있으면 출력, 없으면 content만
    if thinking_content:
        print("주의: thinking content가 남아 있음:", thinking_content)
    
    return content  # 최종 content 반환 (dictionary 예상)

# 5. 3개 데이터로 베이스 모델 generate 실행
for i, text in enumerate(test_texts, 1):
    # print(f"\n===== 테스트 데이터 {i} (카테고리 힌트: 부동산/산업/증권) =====\n{text}\n")
    
    # generate 호출 (모델 로드 후 바로 여기서 generate)
    base_output = generate_output(base, text, system_prompt)
    print("----- 베이스 모델 출력 -----\n", base_output)
    
    # generate 호출 (모델 로드 후 바로 여기서 generate)
    base_output = generate_output(lora, text, system_prompt)
    print("----- 로라 모델 출력 -----\n", base_output)
    
    print(f"\n===== {i}번째 비교 테스트 끝 =====\n")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


----- 베이스 모델 출력 -----
 {"category": "산업", "event_count": 4, "events": ["6·27 대출 규제 시행 후 도봉구 아파트 가격 상승세 유지", "도봉구 실거래가 회복 조짐 나타나고 있다", "금관구 거래량 감소 및 가격 상승세 하락", "부동산업계 전문가가 외곽 지역 시장 흐름의 불확실성 강조"]}


The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

----- 베이스 모델 출력 -----
 답변:
{"category": "산업", "event_count": 4, "events": ["한일시멘트가 자회사 한일현대시멘트를 흡수 합병 결정", "시멘트 산업의 경영상황 악화", "업계 전반의 대응은 신중한 모습", "시멘트 제조 시 온실가스 배출량 증가로 탄소중립 목표 설정"]}


The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


----- 로라 모델 출력 -----
 {"category": "산업","event_count": 2,"events": ["20일 관련 업계에 따르면 한일시멘트는 최근 이사회를 열고 77.78% 지분을 보유한 한일현대시멘트를 오는 11월 1일부로 흡수 합병하기로 의결했다.", "이미 시멘트 업계의 경영상황은 악화일로다."]}

===== 2번째 비교 테스트 끝 =====



The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


----- 베이스 모델 출력 -----
 답변:
{"category": "산업", "event_count": 10, "events": ["인공지능(AI) 분야 경쟁이 '매그니피센트 세븐' 기업 주가에 심화되며, 주가 차별화가 뚜렷해졌습니다.", "Amazon, Alphabet, Apple, Meta, Microsoft, NVIDIA, Tesla의 주가가 AI 성과에 따라 크게 갈리고 있습니다.", "S&P 500 지수에서 7개 기업이 차지하는 비중은 35%에 달하며, 이들 기업의 2분기 순이익이 전년 동기 대비 14% 증가했습니다.", "Alphabet과 Tesla는 주가가 20% 이상 상승했으나, Apple과 Meta는 16% 하락했습니다.", "Amazon은 관세 영향으로 전자상거래 부문에 타격을 받았지만, 주가가 3% 올랐습니다.", "Alphabet과 Tesla는 23일 분기 실적 발표를, Meta, Microsoft, Apple는 다음 주에 발표할 예정입니다.", "이바나 델레브스카는 7개 기업이 AI 분야 선봉에 있는 점을 강조하며, 펀더멘털 차이가 뚜렷하다고 평가했습니다.", "Harry Ferguson의 매니징 파트너인 제이미 콕스는 테크 대기업 간 실적 차이로 주가 흐름이 차별화된 점을 지적했다."}
----- 로라 모델 출력 -----
 {"category": "증권","event_count": 2,"events": ["올해 들어 엔비디아, 메타, MS의 주가는 약 20% 이상 올랐으나, 이와 대조적으로 애플은 16% 하락했고 알파벳도 2% 내렸고 테슬라는 18%가 빠졌다.", "AI 스타트업 '앤스로픽'에 투자한 아마존은 관세 등의 영향으로 전자상거래 부문 사업에 타격이 상당히 큰 가운데서도 3% 올랐다."]}

===== 3번째 비교 테스트 끝 =====



In [7]:
from safetensors.torch import load_file
import torch

path = "./qwen3-lora-ft-final/adapter_model.safetensors"
state = load_file(path)                 # dict[str, torch.Tensor]

# LoRA 파라미터만 골라서 L1-norm 합계 구하기
total = sum(t.abs().sum().item()
            for k, t in state.items()
            if "lora_" in k)

print(f"L1-norm 합: {total:.4f}")
# 0 이면 학습이 안 된 것, 0보다 크면 정상 학습

L1-norm 합: 24584.0718


# 최종 배치 데이터 확인해보기 (배치 1로 변경) (collator 잘 됐는지)

In [24]:
test_training_args = SFTConfig(
    output_dir="./qwen3-lora-ft",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    optim="adamw_torch",
    bf16=True,
    gradient_checkpointing=True,
    logging_steps=50,
    save_steps=50,
    save_total_limit=3,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="linear",
    report_to="none",
    remove_unused_columns=False,
    weight_decay=0.01,
    max_seq_length = 8192,
    packing        = False,
)


# 5) SFTTrainer
test_trainer = SFTTrainer(
    model             = model,
    args              = test_training_args,           # ← SFTConfig 객체
    train_dataset     = tokenized_dataset,
    eval_dataset      = None,
    processing_class  = tokenizer,         # ← tokenizer 자리
)

Truncating train dataset:   0%|          | 0/3265 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [25]:
tokenized_dataset.features

{'text': Value('string'),
 'input_ids': List(Value('int32')),
 'attention_mask': List(Value('int8')),
 'labels': List(Value('int64'))}

In [26]:
# (이미 trainer가 정의된 후) dataloader 가져오기
test_train_dataloader = test_trainer.get_train_dataloader()

# 첫 번째 배치 가져오기 (작은 배치로 테스트)
batch = next(iter(test_train_dataloader))  # dataloader의 첫 배치 추출

# 배치 내용 확인 (input_ids, attention_mask, labels 출력)
print("===== 첫 번째 배치 확인 =====")
print(f"Batch keys: {batch.keys()}")  # 예상: dict_keys(['input_ids', 'attention_mask', 'labels'])

# 각 키의 shape와 샘플 값 출력
for key in ['input_ids', 'attention_mask', 'labels']:
    print(f"\n{key.upper()}:")
    print(f"  Shape: {batch[key].shape}")  # 예: torch.Size([batch_size, sequence_length])
    print(f"  Sample (첫 20개 토큰): {batch[key][0][:20].tolist()}")  # 첫 샘플의 앞부분
    print(f"  Sample (마지막 20개 토큰): {batch[key][0][-20:].tolist()}")  # 첫 샘플의 뒷부분 (패딩 확인)

# labels가 제대로 설정됐는지 추가 체크 (assistant 부분만 학습되는지)
# -100이 아닌 부분이 assistant 답변인지 수동으로 확인하세요!
non_ignore_count = (batch['labels'] != -100).sum().item()
print(f"\n Labels에서 학습되는 토큰 수 (-100 아닌 부분): {non_ignore_count}")

===== 첫 번째 배치 확인 =====
Batch keys: dict_keys(['input_ids', 'attention_mask', 'labels'])

INPUT_IDS:
  Shape: torch.Size([2, 1378])
  Sample (첫 20개 토큰): [151644, 8948, 198, 64795, 82528, 33704, 5140, 231, 112, 24897, 10764, 44104, 53189, 18411, 128618, 129150, 82190, 90711, 112, 130229]
  Sample (마지막 20개 토큰): [151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643]

ATTENTION_MASK:
  Shape: torch.Size([2, 1378])
  Sample (첫 20개 토큰): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  Sample (마지막 20개 토큰): [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

LABELS:
  Shape: torch.Size([2, 1378])
  Sample (첫 20개 토큰): [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
  Sample (마지막 20개 토큰): [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -

In [27]:
# batch 이미 가져온 후 (trainer.get_train_dataloader() 후 next(iter()))
import torch

# labels[0] 전체 출력 (너무 길면 앞뒤 자름, but 전체 확인 추천)
print("===== labels[0] 전체 (또는 일부) =====")
print(batch['labels'][0].tolist())  # 전체 리스트로 출력

# assistant 시작 위치 찾기 (Qwen 토큰 기준, <im_start|>assistant\n)
assistant_start_tokens = [151644, 77091, 198]  # <im_start|>assistant\n 토큰 (tokenizer 확인 필요)
input_ids_list = batch['input_ids'][0].tolist()
labels_list = batch['labels'][0].tolist()

start_idx = -1
for i in range(len(input_ids_list) - len(assistant_start_tokens) + 1):
    if input_ids_list[i:i+len(assistant_start_tokens)] == assistant_start_tokens:
        start_idx = i + len(assistant_start_tokens)  # assistant\n 다음부터
        break

if start_idx != -1:
    # assistant 끝 찾기 (<|im_end|>: 151645)
    end_idx = start_idx
    for j in range(start_idx, len(input_ids_list)):
        if input_ids_list[j] == 151645:  # <|im_end|>
            end_idx = j + 1  # include end token
            break
    else:
        end_idx = len(input_ids_list)  # end 없으면 전체

    print(f"\nassistant 부분 (idx {start_idx} ~ {end_idx}):")
    print("input_ids:", input_ids_list[start_idx:end_idx])
    print("labels:", labels_list[start_idx:end_idx])
    print("같은가? :", input_ids_list[start_idx:end_idx] == labels_list[start_idx:end_idx]) 

    # -100 아닌 개수 다시 체크
    assistant_non_ignore = sum(1 for x in labels_list[start_idx:end_idx] if x != -100)
    print(f"assistant 부분 학습 토큰 수: {assistant_non_ignore}")  # ~81 정도 예상 (배치당)
else:
    print("assistant 시작 못 찾음 - 데이터셋 문제?")

===== labels[0] 전체 (또는 일부) =====
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 

In [36]:
from transformers import AutoTokenizer
tokenizer = test_trainer.tokenizer           # SFTTrainer 가 자동으로 갖고 있습니다

ids = batch["input_ids"][0].tolist()         
txt = tokenizer.decode(ids, skip_special_tokens=False)
print(txt)

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


<|im_start|>system
당신은 뉴스 텍스트를 분석하여 카테고리를 분류하고 주요 핵심 이벤트들을 추출하는 전문 분석 시스템입니다.
주어진 텍스트를 분석하여 반드시 파이썬의 dictionary 형식으로 결과를 작성하십시오.
큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다.
아래 dictionary에서 각 value는 지시사항에 해당합니다. 지시사항을 그대로 적지 마시고, 해당 지시사항에 따라 적절한 value를 채워넣으십시오.

분석 결과는 다음 형식으로 작성하십시오:

답변:
{"category": "텍스트의 카테고리를 ['부동산', '산업', '오피니언', '증권'] 중 하나로 분류하여 작성하십시오",
"event_count": "텍스트 내에서 발견된 핵심 이벤트의 개수를 정수로 작성하십시오(증감, 변화, 변동이 나오는 구절을 중점적으로 보세요.)",
"events": ["텍스트에서 추출한 각 핵심 이벤트를 요약하는 문장들을 파이썬 문자열 리스트 형태로 작성하십시오. 원문의 핵심 내용을 그대로 유지하되 한 문장으로 간결하게 작성하십시오"]}<|im_end|>
<|im_start|>user
다음 뉴스 텍스트를 분석해주세요:

현대자동차가 중국·북미 등 세계시장 공략용 고성능 모델을 잇달아 선보인다. 고급화·전동화에 이어 고성능화에 속도를 냄으로써 브랜드 이미지를 더욱 끌어올리겠다는 전략으로 풀이된다.  3일 자동차 업계에 따르면 현대차는 고성능 N 브랜드 신차인 '더 뉴 아반떼 N(현지명 더 뉴 엘란트라 N)'을 오는 10월 중국, 11월 북미 지역에 잇따라 출시할 계획이다. 한국시장에 내놓는 건 10월이 될 전망이다. 아반떼 N은 지난 3월 출시한 부분변경 차량인 '더 뉴 아반떼'의 고성능 세단 모델이다. 현대차는 지난 4월 열린 상하이 국제모터쇼에서 N 브랜드의 중국시장 진출을 공식 선언한 바 있다. 세계 최대 자동차시장인 중국에서 점유율이 1.7%(작년 기준)에 불과할 정

In [38]:
import torch

dl = test_trainer.get_train_dataloader()
batch = next(iter(dl))              # 아무 배치 하나 뽑기

ids        = batch["input_ids"]         # (B, L)
attn_mask  = batch["attention_mask"]    # (B, L)

# ① 배치 전체 정보
print("input_ids.shape :", ids.shape)
print("real lengths    :", attn_mask.sum(1).tolist())
print("pad lengths     :", (ids.shape[1] - attn_mask.sum(1)).tolist())

# ② 패딩이 진짜 0 인지, 특수 토큰은 제외됐는지
row = 0                        # 보고 싶은 시퀀스 인덱스
print("\n----- raw ids 끝 20개 -----")
print(ids[row, -20:])

print("\n----- attention_mask 끝 20개 -----")
print(attn_mask[row, -20:])

# ③ attention_mask==1 위치만 디코딩
tok = test_trainer.processing_class   # Trainer.tokenizer 대신
real_ids = ids[row][attn_mask[row].bool()]
print(f"\n실제 토큰 수 : {real_ids.size(0)}")

text = tok.decode(real_ids, skip_special_tokens=True)
print(text[:800] + (" …" if len(text) > 800 else ""))

input_ids.shape : torch.Size([2, 1378])
real lengths    : [983, 1378]
pad lengths     : [395, 0]

----- raw ids 끝 20개 -----
tensor([151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
        151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
        151643, 151643], device='cuda:0')

----- attention_mask 끝 20개 -----
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

실제 토큰 수 : 983
system
당신은 뉴스 텍스트를 분석하여 카테고리를 분류하고 주요 핵심 이벤트들을 추출하는 전문 분석 시스템입니다.
주어진 텍스트를 분석하여 반드시 파이썬의 dictionary 형식으로 결과를 작성하십시오.
큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다.
아래 dictionary에서 각 value는 지시사항에 해당합니다. 지시사항을 그대로 적지 마시고, 해당 지시사항에 따라 적절한 value를 채워넣으십시오.

분석 결과는 다음 형식으로 작성하십시오:

답변:
{"category": "텍스트의 카테고리를 ['부동산', '산업', '오피니언', '증권'] 중 하나로 분류하여 작성하십시오",
"event_count": "텍스트 내에서 발견된 핵심 이벤트의 개수를 정수로 작성하십시오(증감, 변화, 변동이 나오는 구절을 중점적으로 보세요.)",
"events": ["텍스트에서 추출한 각 핵심 이벤트를 요약하는 문장들을 파이썬 문자열 리스트 형태로 작성하십시오. 원문의 

In [31]:
import torch

dl = test_trainer.get_train_dataloader()
batch = next(iter(dl))

print("input_ids.shape :", batch["input_ids"].shape)      # (batch, seq_len)
print("최대 길이        :", batch["attention_mask"].sum(1).max().item())
print("배치별 실제 길이 :", batch["attention_mask"].sum(1).tolist())

input_ids.shape : torch.Size([2, 1378])
최대 길이        : 1378
배치별 실제 길이 : [983, 1378]


In [37]:
import torch

tokenizer = test_trainer.tokenizer      # 이미 로드돼 있음
batch     = next(iter(test_trainer.get_train_dataloader()))

for i in range(batch["input_ids"].size(0)):          # 배치(=2개 샘플) 순회
    # 1) attention_mask==1 인 부분만 잘라서 실제 토큰만 남김
    real_ids = batch["input_ids"][i][batch["attention_mask"][i].bool()]

    # 2) 디코딩
    text = tokenizer.decode(real_ids, skip_special_tokens=True)

    # 3) 결과 출력
    print(f"\n─── 샘플 {i} / 실제 길이 {real_ids.size(0)} ───")
    print(text[:8192] + (" …" if len(text) > 8192 else ""))   # 너무 길면 앞 800자만

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.



─── 샘플 0 / 실제 길이 983 ───
system
당신은 뉴스 텍스트를 분석하여 카테고리를 분류하고 주요 핵심 이벤트들을 추출하는 전문 분석 시스템입니다.
주어진 텍스트를 분석하여 반드시 파이썬의 dictionary 형식으로 결과를 작성하십시오.
큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다.
아래 dictionary에서 각 value는 지시사항에 해당합니다. 지시사항을 그대로 적지 마시고, 해당 지시사항에 따라 적절한 value를 채워넣으십시오.

분석 결과는 다음 형식으로 작성하십시오:

답변:
{"category": "텍스트의 카테고리를 ['부동산', '산업', '오피니언', '증권'] 중 하나로 분류하여 작성하십시오",
"event_count": "텍스트 내에서 발견된 핵심 이벤트의 개수를 정수로 작성하십시오(증감, 변화, 변동이 나오는 구절을 중점적으로 보세요.)",
"events": ["텍스트에서 추출한 각 핵심 이벤트를 요약하는 문장들을 파이썬 문자열 리스트 형태로 작성하십시오. 원문의 핵심 내용을 그대로 유지하되 한 문장으로 간결하게 작성하십시오"]}
user
다음 뉴스 텍스트를 분석해주세요:

현대자동차가 중국·북미 등 세계시장 공략용 고성능 모델을 잇달아 선보인다. 고급화·전동화에 이어 고성능화에 속도를 냄으로써 브랜드 이미지를 더욱 끌어올리겠다는 전략으로 풀이된다.  3일 자동차 업계에 따르면 현대차는 고성능 N 브랜드 신차인 '더 뉴 아반떼 N(현지명 더 뉴 엘란트라 N)'을 오는 10월 중국, 11월 북미 지역에 잇따라 출시할 계획이다. 한국시장에 내놓는 건 10월이 될 전망이다. 아반떼 N은 지난 3월 출시한 부분변경 차량인 '더 뉴 아반떼'의 고성능 세단 모델이다. 현대차는 지난 4월 열린 상하이 국제모터쇼에서 N 브랜드의 중국시장 진출을 공식 선언한 바 있다. 세계 최대 자동차시장인 중국에서 점유율이 1.7%(작년 기준)에 불과할 정도로 부진을 겪